<a href="https://colab.research.google.com/github/saigerutherford/CPC_ML_tutorial/blob/master/tasks_key/key_cpc_normative_modeling_instructions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CPC TUTORIAL ON MACHINE LEARNING & NORMATIVE MODELING**


Created by: [Saige Rutherford](https://twitter.com/being_saige) & [Thomas Wolfers](https://twitter.com/ThomasWolfers)
_______________________________________________________________________________

## **Background Story**

Morten and Ingrid are concerned about the health of their father, Nordan. He recently turned 65 years old. A few months ago he could not find his way home. Together, they visit a neurologist/psychiatrist to conduct a number of cognitive tests. However, those tests were inconclusive. While Nordan has a relatively low IQ it could not explain his trouble returning home.

Recently, the family heard about a new screening technique called normative modeling with which one can place individuals in reference to a population norm on for instance measures such as brain volume. Nordan would like to undertake this procedure to better know what is going on and to potentially find targets for treatment. Therefore, the family booked an appointment with you, the normative modeling specialist. To find out what is going on you compare Nordan's hyppocampus to the norm and to a group of persons with Dementia disorders, who have a similar IQ, age as well as the same sex as Nordan.

Do your best to get as far as you can. However, you do not need to feel bad if you cannot complete everything during the tutorial. All materials will stay online and can be accessed after the course. 


## **Task 0:** Load data and install the pcntoolkit 

In [ ]:
# Install the normative modeling python package, predictive clinical neuroscience (pcn) toolkit
# Make sure you use pcntoolkit version 0.20 and python 3.8
!pip install pcntoolkit==0.20

**Option 1:** Connect your Google Drive account, and load data from Google Drive. Having Google Drive connected will allow you to save any files created back to your Drive folder. This step will require you to download the csv files from [Github](https://github.com/saigerutherford/CPC_ML_tutorial/tree/master/data) to your computer, and then make a folder in your Google Drive account and upload the csv files to this folder. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#change dir to data on your google drive
import os
os.chdir('drive/My Drive/name-of-folder-where-you-uploaded-csv-files-from-Github/') #Change this path to match the path to your data in Google Drive

**Option 2 (preferred):** Import the files directly from Github, and skip adding them to Google Drive.

In [ ]:
!wget -nc https://raw.githubusercontent.com/saigerutherford/CPC_ML_tutorial/master/data/cam_demographics.csv
!wget -nc https://raw.githubusercontent.com/saigerutherford/CPC_ML_tutorial/master/data/hcp_demographics_nordan.csv
!wget -nc https://raw.githubusercontent.com/saigerutherford/CPC_ML_tutorial/master/data/ixi_demographics_nordan.csv
!wget -nc https://raw.githubusercontent.com/saigerutherford/CPC_ML_tutorial/master/data/cam_aparc_thickness.csv
!wget -nc https://raw.githubusercontent.com/saigerutherford/CPC_ML_tutorial/master/data/hcp_aparc_thickness.csv
!wget -nc https://raw.githubusercontent.com/saigerutherford/CPC_ML_tutorial/master/data/ixi_aparc_thickness.csv

In [1]:
# import necessary libraries
import os
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

os.chdir('/Users/saigerutherford/repos/CPC_ML_tutorial/')

In [2]:
from pcntoolkit.normative import estimate, predict, evaluate
from pcntoolkit.util.utils import compute_MSLL, create_design_matrix
from nm_utils import calibration_descriptives, remove_bad_subjects, load_2d

## **TASK 1:** Format input data

You have four files. The features and demographics file for the training set (normative sample) and two files of the same name for Nordan (your test sample). As one of your co-workers has done the preporcessing and quality control, there are more subjects in the demographics file than in the features file of the norm sample. Please select the overlap of participants between those two files. 

*Question for your understanding:*

1) Why do we have to select the overlap between participants in terms of features and demographics?

In [ ]:
import joypy
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#create a color gradent function to be used in the colormap parameter
def color_gradient(x=0.0, start=(0, 0, 0), stop=(1, 1, 1)):
    r = np.interp(x, [0, 1], [start[0], stop[0]])
    g = np.interp(x, [0, 1], [start[1], stop[1]])
    b = np.interp(x, [0, 1], [start[2], stop[2]])
    return r, g, b#show the table
#plot the figure
plt.figure(dpi=380)
fig, axes = joypy.joyplot(train_df, column=['age'], overlap=2.5, by="site", ylim='own', fill=True, figsize=(4,4)
                          , legend=False, xlabels=True, ylabels=True, colormap=lambda x: color_gradient(x, start=(.08, .45, .8),stop=(.8, .34, .44))
                          , alpha=0.6, linewidth=.5, linecolor='w', fade=True)
plt.title('Training Set Per Site Age Distribution', fontsize=18, color='black', alpha=1)
plt.xlabel('Age', fontsize=14, color='black', alpha=1)
plt.ylabel('Site', fontsize=14, color='black', alpha=1)
plt.show()

In [ ]:
import joypy
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#create a color gradent function to be used in the colormap parameter
def color_gradient(x=0.0, start=(0, 0, 0), stop=(1, 1, 1)):
    r = np.interp(x, [0, 1], [start[0], stop[0]])
    g = np.interp(x, [0, 1], [start[1], stop[1]])
    b = np.interp(x, [0, 1], [start[2], stop[2]])
    return r, g, b#show the table
#plot the figure
plt.figure(dpi=380)
fig, axes = joypy.joyplot(test_df, column=['age'], overlap=2.5, by="site", ylim='own', fill=True, figsize=(4,4)
                          , legend=False, xlabels=True, ylabels=True, colormap=lambda x: color_gradient(x, start=(.08, .45, .8),stop=(.8, .34, .44))
                          , alpha=0.7, linewidth=.5, linecolor='w', fade=True)
plt.title('Test Set Per Site Age Distribution', fontsize=18, color='black', alpha=1)
plt.xlabel('Age', fontsize=14, color='black', alpha=1)
plt.ylabel('Site', fontsize=14, color='black', alpha=1)
plt.show()

In [ ]:
import joypy
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
#create a color gradent function to be used in the colormap parameter
def color_gradient(x=0.0, start=(0, 0, 0), stop=(1, 1, 1)):
    r = np.interp(x, [0, 1], [start[0], stop[0]])
    g = np.interp(x, [0, 1], [start[1], stop[1]])
    b = np.interp(x, [0, 1], [start[2], stop[2]])
    return r, g, b#show the table
#plot the figure
plt.figure(dpi=380)
fig, axes = joypy.joyplot(test_df, column=['age'], overlap=2.5, by="site", ylim='own', fill=True, figsize=(4,4)
                          , legend=False, xlabels=True, ylabels=True, colormap=lambda x: color_gradient(x, start=(.08, .45, .8),stop=(.8, .34, .44))
                          , alpha=0.7, linewidth=.5, linecolor='w', fade=True)
plt.title('Transfer Set Per Site Age Distribution', fontsize=18, color='black', alpha=1)
plt.xlabel('Age', fontsize=14, color='black', alpha=1)
plt.ylabel('Site', fontsize=14, color='black', alpha=1)
plt.show()

In [3]:
# read in the files.
train_covariates = pd.read_csv('data/train_covariates.csv')
train_features = pd.read_csv('data/train_features.csv')

# find overlap in terms of participants between the train features and covariates dataframes
train_df = pd.merge(train_covariates, train_features, how = 'inner') # inner checks overlap

In [4]:
# read in the files.
test_covariates = pd.read_csv('data/test_covariates.csv')
test_features = pd.read_csv('data/test_features.csv')

# find overlap in terms of participants between the train features and covariates dataframes
test_df = pd.merge(test_covariates, test_features, how = 'inner') # inner checks overlap

In [ ]:
# read in the files.
transfer_covariates = pd.read_csv('data/transfer_covariates.csv')
transfer_features = pd.read_csv('data/transfer_features.csv')

# find overlap in terms of participants between the train features and covariates dataframes
transfer_df = pd.merge(transfer_covariates, transfer_features, how = 'inner') # inner checks overlap

In [5]:
train_df.shape

(1163, 74)

In [6]:
test_df.shape

(582, 74)

In [ ]:
transfer_df.shape

## **TASK 2:** Prepare the covariate + response files and specify model parameters.

As mentioned in the introductory presentation those files need a specific format and the entries need to be seperated by spaces. Use whatever method you know to prepare those files based on the data provided in TASK 1. Save those files in .txt format in your drive. Also get rid of the column names and participant IDs.

Given that we only have limited time in this practical we have to make a selection for the features based on your prior knowledge. With the information in mind that Nordan does not remember his way home, which subfield of the hyppocampus is probably a good target for the investigations?
Select a maximum of four hyppocampal regions as features.

NOTE: Normative modeling is a screening tool we just make this selection due to time constraints, in reality we build these models on millions of putative biomarkers that are not restricted to brain imaging.


*Questions for your understanding:*

2) What is the requirement for the features in terms of variable properties (e.g. dicotomous or continous)? 

3) What is the requirement for the covariates in terms of these properties? 

4) What are the requirements for both together? 

5) How does this depend on the algorithm used?

In [12]:
# perpare covariate_normsample for sex and age
covariate_train = train_df[cov_cols]

# perpare features_normsample for relevant hyppocampal subfields
features_train = train_df[feature_cols]

# extract a list of unique site ids from the training set
site_ids_tr =  sorted(set(train_df['site'].to_list()))

In [7]:
column_names = train_df.columns.to_list()

In [8]:
cov_cols = ['age', 'sex', 'site']

In [9]:
feature_cols = [i for i in column_names if i not in cov_cols]

In [10]:
print(feature_cols)

['participant_id', 'lh_bankssts_thickness', 'lh_caudalanteriorcingulate_thickness', 'lh_caudalmiddlefrontal_thickness', 'lh_cuneus_thickness', 'lh_entorhinal_thickness', 'lh_fusiform_thickness', 'lh_inferiorparietal_thickness', 'lh_inferiortemporal_thickness', 'lh_isthmuscingulate_thickness', 'lh_lateraloccipital_thickness', 'lh_lateralorbitofrontal_thickness', 'lh_lingual_thickness', 'lh_medialorbitofrontal_thickness', 'lh_middletemporal_thickness', 'lh_parahippocampal_thickness', 'lh_paracentral_thickness', 'lh_parsopercularis_thickness', 'lh_parsorbitalis_thickness', 'lh_parstriangularis_thickness', 'lh_pericalcarine_thickness', 'lh_postcentral_thickness', 'lh_posteriorcingulate_thickness', 'lh_precentral_thickness', 'lh_precuneus_thickness', 'lh_rostralanteriorcingulate_thickness', 'lh_rostralmiddlefrontal_thickness', 'lh_superiorfrontal_thickness', 'lh_superiorparietal_thickness', 'lh_superiortemporal_thickness', 'lh_supramarginal_thickness', 'lh_frontalpole_thickness', 'lh_tempor

In [11]:
feature_cols.remove('participant_id')

In [13]:
# perpare covariate_normsample for sex and age
covariate_test = test_df[cov_cols]

# perpare features_normsample for relevant hyppocampal subfields
features_test = test_df[feature_cols]

# extract a list of unique site ids from the test set
site_ids_te =  sorted(set(test_df['site'].to_list()))

In [16]:
# which warping function to use? We can set this to None in order to fit a vanilla Gaussian noise model.
warp =  'WarpSinArcsinh'

# limits for cubic B-spline basis
xmin = 15
xmax = 85

# Do we want to force the model to be refit every time? 
# Set this to False if you want to transfer a pre-trained model to new sites not included in the training sample
force_refit = True

# Absolute Z treshold above which a sample is considered to be an outlier (without fitting any model). 
# This is an attempt at automated QC. It should be a really big threshold and ideally you would visually 
# inspect images that meet this criteria to confirm it is due to poor data quality i.e., motion or artifact.
outlier_thresh = 7

In [ ]:
# perpare covariate_normsample for sex and age
covariate_transfer = transfer_df[cov_cols]

# perpare features_normsample for relevant hyppocampal subfields
features_transfer = transfer_df[feature_cols]

# extract a list of unique site ids from the test set
site_ids_tf =  sorted(set(transfer_df['site'].to_list()))

In [14]:
print(covariate_train.shape)
print(covariate_test.shape)
#print(covariate_transfer.shape)

(1163, 3)
(582, 3)


In [15]:
print(features_train.shape)
print(features_test.shape)
#print(features_transfer.shape)

(1163, 70)
(582, 70)


## **TASK 3:** Estimate normative model


Once you have prepared and saved all the necessary files. Look at the pcntoolkit for running normative modeling. Select an appropritate method set up the toolkit and run your analyses using 2-fold cross validation in the normsample. 

HINT: You primarily need the estimate function. 

*Question for your understaning:*

6) What does cvfolds mean and why do we use it? 

7) What is the output of the estimate function and what does it mean?

In [17]:
# where the analysis takes place
root_dir = '/Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key'
out_dir = os.path.join(root_dir,'models','test')

# create the output directory if it does not already exist
os.makedirs(out_dir, exist_ok=True)

In [18]:
for roi_num, roi in enumerate(features_train):
    print('Running ROI', roi_num, roi, ':')

    # set output dir
    roi_dir = os.path.join(out_dir, roi)
    os.makedirs(os.path.join(roi_dir), exist_ok=True)
    os.chdir(roi_dir)

    # extract the response variables for training and test set
    y_tr = features_train[roi].to_numpy()
    y_te = features_test[roi].to_numpy()

    # remove gross outliers and implausible values
    yz_tr = (y_tr - np.mean(y_tr)) / np.std(y_tr)
    yz_te = (y_te - np.mean(y_te)) / np.std(y_te)
    nz_tr = np.bitwise_and(np.abs(yz_tr) < outlier_thresh, y_tr > 0)
    nz_te = np.bitwise_and(np.abs(yz_te) < outlier_thresh, y_te > 0)
    y_tr = y_tr[nz_tr]
    y_te = y_te[nz_te]

    # write out the response variables for training and test
    resp_file_tr = os.path.join(roi_dir, 'resp_tr.txt')
    resp_file_te = os.path.join(roi_dir, 'resp_te.txt')
    np.savetxt(resp_file_tr, y_tr)
    np.savetxt(resp_file_te, y_te)

    # configure the design matrix
    X_tr = create_design_matrix(features_train.loc[nz_tr],
                                site_ids = covariate_train['site'].loc[nz_tr],
                                basis = 'bspline',
                                xmin = xmin,
                                xmax = xmax)
    X_te = create_design_matrix(features_test.loc[nz_te],
                                site_ids = covariate_test['site'].loc[nz_te],
                                all_sites=site_ids_tr,
                                basis = 'bspline',
                                xmin = xmin,
                                xmax = xmax)

    # configure and save the covariates
    cov_file_tr = os.path.join(roi_dir, 'cov_bspline_tr.txt')
    cov_file_te = os.path.join(roi_dir, 'cov_bspline_te.txt')
    np.savetxt(cov_file_tr, X_tr)
    np.savetxt(cov_file_te, X_te)

    if not force_refit and os.path.exists(os.path.join(roi_dir, 'Models', 'NM_0_0_estimate.pkl')):
        print('Making predictions using a pre-existing model...')
        suffix = 'predict'

        # Make prdictsion with test data
        predict(cov_file_te,
                alg='blr',
                respfile=resp_file_te,
                model_path=os.path.join(idp_dir,'Models'), outputsuffix=suffix)
    else:
        print('Estimating the normative model...')
        suffix = 'estimate'
        estimate(cov_file_tr, resp_file_tr, testresp=resp_file_te,
                 testcov=cov_file_te, alg='blr', optimizer = 'l-bfgs-b',
                 savemodel=True, warp=warp, warp_reparam=True, outputsuffix=suffix)

Running ROI 0 lh_bankssts_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/lh_bankssts_thickness/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 1 lh_caudalanteriorcingulate_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/lh_caudalanteriorcingulate_thickness/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 2 lh_caudalmiddlefrontal_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/lh_caudalmiddlefrontal_thickness/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using defau

/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=3.55792e-19): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=4.32308e-19): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.05934e-18): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=6.35054e-19): result may not be accurate.
  invAXt = linalg.sol

Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 7 lh_inferiortemporal_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/lh_inferiortemporal_thickness/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 8 lh_isthmuscingulate_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/lh_isthmuscingulate_thickness/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 9 lh_lateraloccipital_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/lh_lateraloccipital_thickness/resp_tr.t

/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=3.43057e-24): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=3.10411e-24): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=4.19011e-24): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=3.79137e-24): result may not be accurate.
  invAXt = linalg.sol

Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 26 lh_superiorfrontal_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/lh_superiorfrontal_thickness/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 27 lh_superiorparietal_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/lh_superiorparietal_thickness/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 28 lh_superiortemporal_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/lh_superiortemporal_thickness/resp_tr.

...

Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 30 lh_frontalpole_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/lh_frontalpole_thickness/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 31 lh_temporalpole_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/lh_temporalpole_thickness/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 32 lh_transversetemporal_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/lh_transversetemporal_thickness/resp_tr.txt
Estimati

/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=2.09396e-225): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.89469e-225): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=2.55757e-225): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=2.31418e-225): result may not be accurate.
  invAXt = linalg

Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 42 rh_inferiortemporal_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/rh_inferiortemporal_thickness/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 43 rh_isthmuscingulate_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/rh_isthmuscingulate_thickness/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 44 rh_lateraloccipital_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/rh_lateraloccipital_thickness/resp_t

/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=3.32601e-19): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=4.35213e-20): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=8.67554e-19): result may not be accurate.
  invAXt = linalg.solve(self.A, X.T, check_finite=False)
/opt/anaconda3/envs/cpc_ml/lib/python3.8/site-packages/pcntoolkit-0.20-py3.8.egg/pcntoolkit/model/bayesreg.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=3.5967e-19): result may not be accurate.
  invAXt = linalg.solv

Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 58 rh_precuneus_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/rh_precuneus_thickness/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 59 rh_rostralanteriorcingulate_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/rh_rostralanteriorcingulate_thickness/resp_tr.txt
Estimating model  1 of 1
configuring BLR ( order 1 )
Using default hyperparameters
Saving model meta-data...
Evaluating the model ...
Writing outputs ...
Running ROI 60 rh_rostralmiddlefrontal_thickness :
Estimating the normative model...
Processing data in /Users/saigerutherford/repos/CPC_ML_tutorial/tasks_key/models/test/rh_rostralmiddlefrontal_thickn

## **TASK 4:** Evaluate the model performance (compute error metrics)

In this section we compute the following error metrics for all ROIs (all evaluated on the test set):

- Negative log likelihood (NLL)

- Explained variance (EV)

- Mean standardized log loss (MSLL)

- Bayesian information Criteria (BIC)

- Skew and Kurtosis of the Z-distribution

In [19]:
# initialise dataframe we will use to store quantitative metrics
blr_metrics = pd.DataFrame(columns = ['ROI', 'NLL', 'EV', 'MSLL', 'BIC','Skew','Kurtosis'])

for roi_num, roi in enumerate(features_train):
    roi_dir = os.path.join(out_dir, roi)

    # load the predictions and true data. We use a custom function that ensures 2d arrays
    # equivalent to: y = np.loadtxt(filename); y = y[:, np.newaxis]
    yhat_te = load_2d(os.path.join(roi_dir, 'yhat_' + suffix + '.txt'))
    s2_te = load_2d(os.path.join(roi_dir, 'ys2_' + suffix + '.txt'))
    y_te = load_2d(os.path.join(roi_dir, 'resp_te.txt'))

    with open(os.path.join(roi_dir,'Models', 'NM_0_0_estimate.pkl'), 'rb') as handle:
        nm = pickle.load(handle)

    # compute error metrics
    if warp is None:
        metrics = evaluate(y_te, yhat_te)

        # compute MSLL manually as a sanity check
        y_tr_mean = np.array( [[np.mean(y_tr)]] )
        y_tr_var = np.array( [[np.var(y_tr)]] )
        MSLL = compute_MSLL(y_te, yhat_te, s2_te, y_tr_mean, y_tr_var)
    else:
        warp_param = nm.blr.hyp[1:nm.blr.warp.get_n_params()+1]
        W = nm.blr.warp

        # warp predictions
        med_te = W.warp_predictions(np.squeeze(yhat_te), np.squeeze(s2_te), warp_param)[0]
        med_te = med_te[:, np.newaxis]

        # evaluation metrics
        metrics = evaluate(y_te, med_te)

        # compute MSLL manually
        y_te_w = W.f(y_te, warp_param)
        y_tr_w = W.f(y_tr, warp_param)
        y_tr_mean = np.array( [[np.mean(y_tr_w)]] )
        y_tr_var = np.array( [[np.var(y_tr_w)]] )
        MSLL = compute_MSLL(y_te_w, yhat_te, s2_te, y_tr_mean, y_tr_var)

    Z = np.loadtxt(os.path.join(roi_dir, 'Z_' + suffix + '.txt'))
    [skew, sdskew, kurtosis, sdkurtosis, semean, sesd] = calibration_descriptives(Z)

    BIC = len(nm.blr.hyp) * np.log(y_tr.shape[0]) + 2 * nm.neg_log_lik

    blr_metrics.loc[len(blr_metrics)] = [roi, nm.neg_log_lik, metrics['EXPV'][0],
                                         MSLL[0], BIC, skew, kurtosis]

display(blr_metrics)

#blr_metrics.to_pickle(os.path.join(out_dir,'blr_metrics.pkl'))
blr_metrics.to_csv(os.path.join(out_dir,'blr_metrics.csv'))

,ROI,NLL,EV,MSLL,BIC,Skew,Kurtosis
0,lh_bankssts_thickness,-1536.532087,0.951048,-1.360186,-3044.832581,0.624536,1.189489
1,lh_caudalanteriorcingulate_thickness,-1450.226462,0.973466,-2.289934,-2872.221333,0.877183,2.108007
2,lh_caudalmiddlefrontal_thickness,-1571.672430,0.938149,-1.543253,-3115.113269,0.531500,0.618271
3,lh_cuneus_thickness,-1617.789359,0.965260,-10.821772,-3207.347127,0.667435,1.162752
4,lh_entorhinal_thickness,-1347.637490,0.978687,-34.493279,-2667.043388,0.644184,4.216713
...,...,...,...,...,...,...,...
65,rh_frontalpole_thickness,-1412.285572,0.977919,-5.139689,-2796.339552,0.867501,3.595441
66,rh_temporalpole_thickness,-1324.759419,0.984490,-68.415625,-2621.287247,0.160652,1.790231
67,rh_transversetemporal_thickness,-1433.361835,0.979707,-3.470125,-2838.492078,1.212676,3.740658
68,rh_insula_thickness,-1495.184018,0.950543,-9.546466,-2962.136444,0.483604,1.705491


## **TASK 6:** Visualize the results

In [20]:
# which sex do we want to plot?
sex = 1 # 1 = male 0 = female
if sex == 1:
    clr = 'blue';
else:
    clr = 'red'

# create dummy data for visualisation
print('configuring dummy data ...')
xx = np.arange(xmin, xmax, 0.5)
X0_dummy = np.zeros((len(xx), 2))
X0_dummy[:,0] = xx
X0_dummy[:,1] = sex

# create the design matrix
X_dummy = create_design_matrix(X0_dummy, xmin=xmin, xmax=xmax, site_ids=None, all_sites=site_ids_tr)

# save the dummy covariates
cov_file_dummy = os.path.join(out_dir,'cov_bspline_dummy_mean.txt')
np.savetxt(cov_file_dummy, X_dummy)

configuring dummy data ...


In [28]:
sns.set(style='whitegrid')

for roi_num, roi in enumerate(features_train):
    print('Running ROI', roi_num, roi, ':')
    roi_dir = os.path.join(out_dir, roi)
    os.chdir(roi_dir)

    # load the true data points
    yhat_te = load_2d(os.path.join(roi_dir, 'yhat_estimate.txt'))
    s2_te = load_2d(os.path.join(roi_dir, 'ys2_estimate.txt'))
    y_te = load_2d(os.path.join(roi_dir, 'resp_te.txt'))

    # set up the covariates for the dummy data
    print('Making predictions with dummy covariates (for visualisation)')
    yhat, s2 = predict(cov_file_dummy,
                       alg = 'blr',
                       respfile = None,
                       model_path = os.path.join(roi_dir,'Models'),
                       outputsuffix = '_dummy')

    # load the normative model
    with open(os.path.join(roi_dir,'Models', 'NM_0_0_estimate.pkl'), 'rb') as handle:
        nm = pickle.load(handle)

    # get the warp and warp parameters
    W = nm.blr.warp
    warp_param = nm.blr.hyp[1:nm.blr.warp.get_n_params()+1]

    # first, we warp predictions for the true data and compute evaluation metrics
    med_te = W.warp_predictions(np.squeeze(yhat_te), np.squeeze(s2_te), warp_param)[0]
    med_te = med_te[:, np.newaxis]
    print('metrics:', evaluate(y_te, med_te))

    # then, we warp dummy predictions to create the plots
    med, pr_int = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param)

    # extract the different variance components to visualise
    beta, junk1, junk2 = nm.blr._parse_hyps(nm.blr.hyp, X_dummy)
    s2n = 1/beta # variation (aleatoric uncertainty)
    s2s = s2-s2n # modelling uncertainty (epistemic uncertainty)

    # plot the data points
    y_te_rescaled_all = np.zeros_like(y_te)
    for sid, site in enumerate(site_ids_te):
        # plot the true test data points
        if all(elem in site_ids_tr for elem in site_ids_te):
            # all data in the test set are present in the training set

            # first, we select the data points belonging to this particular site
            idx = np.where(np.bitwise_and(X_te[:,2] == sex, X_te[:,sid+len(cols_cov)+1] !=0))[0]
            if len(idx) == 0:
                print('No data for site', sid, site, 'skipping...')
                continue

            # then directly adjust the data
            idx_dummy = np.bitwise_and(X_dummy[:,1] > X_te[idx,1].min(), X_dummy[:,1] < X_te[idx,1].max())
            y_te_rescaled = y_te[idx] - np.median(y_te[idx]) + np.median(med[idx_dummy])

    # plot the median of the dummy data
    plt.plot(xx, med, clr)

    # fill the gaps in between the centiles
    junk, pr_int25 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.05,0.95])
    junk, pr_int99 = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2), warp_param, percentiles=[0.01,0.99])
    plt.fill_between(xx, pr_int25[:,0], pr_int25[:,1], alpha = 0.1,color=clr)
    plt.fill_between(xx, pr_int95[:,0], pr_int95[:,1], alpha = 0.1,color=clr)
    plt.fill_between(xx, pr_int99[:,0], pr_int99[:,1], alpha = 0.1,color=clr)

    # make the width of each centile proportional to the epistemic uncertainty
    junk, pr_int25l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.05,0.95])
    junk, pr_int99l = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2-0.5*s2s), warp_param, percentiles=[0.01,0.99])
    junk, pr_int25u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.25,0.75])
    junk, pr_int95u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.05,0.95])
    junk, pr_int99u = W.warp_predictions(np.squeeze(yhat), np.squeeze(s2+0.5*s2s), warp_param, percentiles=[0.01,0.99])
    plt.fill_between(xx, pr_int25l[:,0], pr_int25u[:,0], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int95l[:,0], pr_int95u[:,0], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int99l[:,0], pr_int99u[:,0], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int25l[:,1], pr_int25u[:,1], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int95l[:,1], pr_int95u[:,1], alpha = 0.3,color=clr)
    plt.fill_between(xx, pr_int99l[:,1], pr_int99u[:,1], alpha = 0.3,color=clr)

    # plot actual centile lines
    plt.plot(xx, pr_int25[:,0],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int25[:,1],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int95[:,0],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int95[:,1],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int99[:,0],color=clr, linewidth=0.5)
    plt.plot(xx, pr_int99[:,1],color=clr, linewidth=0.5)

    plt.xlabel('Age')
    plt.ylabel(roi)
    plt.title(roi)
    plt.xlim((0,90))
    plt.savefig(os.path.join(roi_dir, 'centiles_' + str(sex)),  bbox_inches='tight')
    plt.show()

Running ROI 0 lh_bankssts_thickness :
Making predictions with dummy covariates (for visualisation)
Loading data ...
Prediction by model  1 of 1


ValueError: shapes (140,13) and (81,) not aligned: 13 (dim 1) != 81 (dim 0)

In [24]:
X_dummy.shape

(140, 13)

70